In [1]:
import sys
import time

sys.path.append("..")
import pandas as pd
import solv_fzcp as sfzcp
import uvarprob as uvpr

In [2]:
test_file = "../test_casado2.csv"


In [3]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col='name', comment='#')
    return data

df = read_problems(test_file)
df

,objective,a,b,min_f,min_x
name,,,,,
1,-0.5*x^2*(log(x))+5,0.2,7.0,0,0
3,-(sqrt(x))*sin(x)+1,0.2,7.0,0,0
4,x*sin(x)+sin(10.0*x/3.0)+log(x)-0.84*x+1.3,0.2,7.0,0,0
5,x+sin(5.0*x),0.2,7.0,0,0
7,sin(x)*cos(x)-1.5*(sin(x))^2+1.2,0.2,7.0,0,0
9,2.0*sin(x)*exp(-x),0.2,7.0,0,0
10,(3*x-1.4)*sin(18.0*x)+1.7,0.2,7.0,0,0
11,(x+1)^3/x^2-7.1,0.2,7.0,0,0
13,12+cos(2.0*x+1)+2*cos(3.0*x+2)+3*cos(4.0*x+3)...,0.2,7.0,0,0


In [4]:
points_db = {}
psl_lipint_points_list = []
psl_lip_points_list = []
psqe_lipint_points_list = []
psqe_lip_points_list = []
bnb2_lipint_points_list = []
bnb2_lip_points_list = []

In [5]:
def log_point(x, points_list):
    points_list.append(x)

In [8]:
import ia_math_fun as iaf
import interval_arithmetics as ia
for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    # #
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
    #                           lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']))
    # period = 0
    # T1 = time.perf_counter()
    # print('Piyavskii, Lipschitz interval:',
    #       sfzcp.correctly(prob, symm=False, epsilon=1e-10, global_lipschitz_interval=False, estimator=1, reduction=1,
    #                        period_comp_lip=period))
    # T2 = time.perf_counter()
    # print('time=', (T2 - T1)*1000, 'period:', period)
    #
    iaf.set_log_taylor_terms_number(19)
    iaf.set_cos_taylor_terms_number(19)
    iaf.set_sin_taylor_terms_number(19)
    iaf.set_exp_taylor_terms_number(19)
    ia.set_precision(19)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=2, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)
    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=1, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

1
PSQE: TestResult(nsteps=3, first_crossing_zero_point=[[3.011688528945636903, 3.011690771062112775]])
time= 13.356292038224638 period: 0
Linear: TestResult(nsteps=5, first_crossing_zero_point=[[3.011686222150624367, 3.011690785574816752]])
time= 11.006874963641167 period: 0
3
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[1.174796171291449297, 1.174796171291492291]])
time= 77.69837498199195 period: 0
Linear: TestResult(nsteps=8, first_crossing_zero_point=[[1.174796064342206803, 1.174796425708054015]])
time= 49.14254101458937 period: 0
4
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[2.960913794435176517, 2.960913794442501751]])
time= 157.11629099678248 period: 0
Linear: TestResult(nsteps=10, first_crossing_zero_point=[[2.960913794415875057, 2.960913794480238457]])
time= 136.92862505558878 period: 0
5
PSQE: TestResult(nsteps=9, first_crossing_zero_point=[[0.8209239701115810884, 0.8209239701115810941]])
time= 51.879583043046296 period: 0
Linear: TestResult(nsteps=10, fir

In [10]:
import ia_math_fun as iaf
import interval_arithmetics as ia
for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    iaf.set_log_taylor_terms_number(19)
    iaf.set_cos_taylor_terms_number(19)
    iaf.set_sin_taylor_terms_number(19)
    iaf.set_exp_taylor_terms_number(19)
    ia.set_precision(19)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=2, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)
    
    
    iaf.set_log_taylor_terms_number(9)
    iaf.set_cos_taylor_terms_number(9)
    iaf.set_sin_taylor_terms_number(9)
    iaf.set_exp_taylor_terms_number(9)
    ia.set_precision(9)
    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=1, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

1
PSQE: TestResult(nsteps=3, first_crossing_zero_point=[[3.011688528945636903, 3.011690771062112775]])
time= 13.515874976292253 period: 0
Linear: TestResult(nsteps=5, first_crossing_zero_point=[[3.01168617, 3.01169087]])
time= 6.573165999725461 period: 0
3
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[1.174796171291449297, 1.174796171291492291]])
time= 82.15316606219858 period: 0
Linear: TestResult(nsteps=8, first_crossing_zero_point=[[1.17479606, 1.17479644]])
time= 18.98870791774243 period: 0
4
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[2.960913794435176517, 2.960913794442501751]])
time= 156.35225002188236 period: 0
Linear: TestResult(nsteps=10, first_crossing_zero_point=[[2.96091375, 2.96091383]])
time= 53.09837497770786 period: 0
5
PSQE: TestResult(nsteps=9, first_crossing_zero_point=[[0.8209239701115810884, 0.8209239701115810941]])
time= 53.084708051756024 period: 0
Linear: TestResult(nsteps=10, first_crossing_zero_point=[[0.820923955, 0.820924025]])
time= 14.

In [11]:
import ia_math_fun as iaf
import interval_arithmetics as ia
for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    iaf.set_log_taylor_terms_number(9)
    iaf.set_cos_taylor_terms_number(9)
    iaf.set_sin_taylor_terms_number(9)
    iaf.set_exp_taylor_terms_number(9)
    ia.set_precision(9)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=2, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)
    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=1, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

1
PSQE: TestResult(nsteps=255, first_crossing_zero_point=[[3.01057353, 3.01058249], [3.01058249, 3.01059144], [3.01059144, 3.01060039], [3.01060039, 3.01060934], [3.01060934, 3.01061829], [3.01061829, 3.01062724], [3.01062724, 3.01063619], [3.01063619, 3.01064514], [3.01064514, 3.01065409], [3.01065409, 3.01066304], [3.01066304, 3.01067199], [3.01067199, 3.01068094], [3.01068094, 3.01068989], [3.01068989, 3.01069884], [3.01069884, 3.01070779], [3.01070779, 3.01071674], [3.01071674, 3.01072569], [3.01072569, 3.01073464], [3.01073464, 3.01074359], [3.01074359, 3.01075254], [3.01075254, 3.01076149], [3.01076149, 3.01077044], [3.01077044, 3.01077939], [3.01077939, 3.01078834], [3.01078834, 3.01079729], [3.01079729, 3.01080624], [3.01080624, 3.01081519], [3.01081519, 3.01082414], [3.01082414, 3.01083309], [3.01083309, 3.01084204], [3.01084204, 3.01085099], [3.01085099, 3.01085994], [3.01085994, 3.01086889], [3.01086889, 3.01087784], [3.01087784, 3.01088679], [3.01088679, 3.01089574], [3.010